<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-lead-ids-by-clientid" data-toc-modified-id="Get-lead-ids-by-clientid-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get lead ids by clientid</a></span></li><li><span><a href="#Get-quoters-Q&amp;As-by-clientid" data-toc-modified-id="Get-quoters-Q&amp;As-by-clientid-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get quoters Q&amp;As by clientid</a></span></li></ul></div>

In [1]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
import httplib2

# Get lead ids by clientid

In [2]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = '197069469' # argentina main

def initialize_analyticsreporting():
    """Initializes an Analytics Reporting API V4 service object.

    Returns:
      An authorized Analytics Reporting API V4 service object.
    """
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

def get_report(analytics, pageToken=None):
    """Queries the Analytics Reporting API V4.

    Args:
      analytics: An authorized Analytics Reporting API V4 service object.
    Returns:
      The Analytics Reporting API V4 response.
    """
    return analytics.reports().batchGet(
        body={
          'reportRequests': [
          {
              'viewId': VIEW_ID,
              'pageToken': pageToken,
              'pageSize': '10000',
              'dateRanges': [{'startDate': '2021-07-07', 'endDate': '2021-12-31'}],
              'metrics': [{'expression': 'ga:uniqueEvents'}],
              'dimensions': [
                  {'name': 'ga:date'},
                  {'name': 'ga:dimension11'},
                  {'name': 'ga:dimension1'}
                  ]
              }], 'useResourceQuotas': True
            }
        ).execute() 

def parse_response(response):
    """Parses and prints the Analytics Reporting API V4 response.

    Args:
      response: An Analytics Reporting API V4 response.
      """
    alist = []
    
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
        
        for row in rows:
            adict = {}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])
            
            for header, dimension in zip(dimensionHeaders, dimensions):
                adict[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    adict[metric.get('name')] = value
                    
            alist.append(adict)
      
    df = pd.DataFrame(alist)
    return df 

def pagination():
    analytics = initialize_analyticsreporting()
    response = get_report(analytics)
    pageToken = response['reports'][0].get('nextPageToken')
    result = pd.DataFrame(parse_response(response))

    while pageToken != None:

        print(pageToken)
        print("still running")
        analytics = initialize_analyticsreporting()
        response = get_report(analytics, str(int(pageToken)+1))
        pageToken = response['reports'][0].get('nextPageToken')
        temp = pd.DataFrame(parse_response(response))

        result = pd.concat([result,temp], axis=0)
        
    return result

if __name__ == '__main__':
    df = pagination()

10000
still running
20001
still running
30002
still running
40003
still running
50004
still running
60005
still running
70006
still running


In [3]:
df_leads = df.reset_index(drop=True)
df_leads

,ga:date,ga:dimension11,ga:dimension1,ga:uniqueEvents
0,20210707,162559549757326,1796177990.1625595499,1
1,20210707,162559815703238,1680934699.1625598157,1
2,20210707,162562667992520,1990645011.1625626682,1
3,20210707,162562677924885,1922511147.1625626779,1
4,20210707,162562682526032,2822540.1614454503,1
...,...,...,...,...
78306,20211231,164100334152529,1808261425.1641003327,1
78307,20211231,164100403725164,1208938549.1600335399,1
78308,20211231,164100411161519,1498184985.1641004110,1
78309,20211231,164100425750682,1188319175.1629584584,1


In [4]:
df_leads.to_csv('data_files/scorer_ga_leads_by_cid.csv', index=False)

# Get quoters Q&As by clientid

In [6]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = '197069469' # argentina main

def initialize_analyticsreporting():
    """Initializes an Analytics Reporting API V4 service object.

    Returns:
      An authorized Analytics Reporting API V4 service object.
    """
    credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics

def get_report(analytics, pageToken=None):
    """Queries the Analytics Reporting API V4.

    Args:
      analytics: An authorized Analytics Reporting API V4 service object.
    Returns:
      The Analytics Reporting API V4 response.
    """
    return analytics.reports().batchGet(
        body={
          'reportRequests': [
          {
              'viewId': VIEW_ID,
              'pageToken': pageToken,
              'pageSize': '10000', # v4 api needs to go 10K on 10K
              'dateRanges': [{'startDate': '2021-07-07', 'endDate': '2021-12-31'}],
              'segments':  [{'segmentId': 'gaid::to3OurFWTsKqBmfoR0-8Iw'}], # filters to only sessions ending on a lead
              'metrics': [{'expression': 'ga:uniqueEvents'}],
              'dimensions': [
                  {'name': 'ga:date'},
                  {'name': 'ga:dimension1'},
                  {'name': 'ga:eventCategory'},
                  {'name': 'ga:eventAction'},
                  {'name': 'ga:eventLabel'},
                  {'name': 'ga:segment'}
                  ],
              'dimensionFilterClauses': [
                  {
                      'filters': [
                          {
                              'dimensionName': 'ga:eventCategory',
                              'operator': 'REGEXP',
                              'expressions': ['Home Funnel|Business Funnel']
                              }
                          ]
                      }
                  ]
              }], 'useResourceQuotas': True
            }
        ).execute() 

def parse_response(response):
    """Parses and prints the Analytics Reporting API V4 response.

    Args:
      response: An Analytics Reporting API V4 response.
      """
    alist = []
    
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])
        
        for row in rows:
            adict = {}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])
            
            for header, dimension in zip(dimensionHeaders, dimensions):
                adict[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    adict[metric.get('name')] = value
                    
            alist.append(adict)
      
    df = pd.DataFrame(alist)
    return df 

def pagination():
    analytics = initialize_analyticsreporting()
    response = get_report(analytics)
    pageToken = response['reports'][0].get('nextPageToken')
    result = pd.DataFrame(parse_response(response))

    while pageToken != None:

        print(pageToken)
        print("still running")
        analytics = initialize_analyticsreporting()
        response = get_report(analytics, str(int(pageToken)+1))
        pageToken = response['reports'][0].get('nextPageToken')
        temp = pd.DataFrame(parse_response(response))

        result = pd.concat([result,temp], axis=0)
        
    return result

if __name__ == '__main__':
    df = pagination()

10000
still running
20001
still running
30002
still running
40003
still running
50004
still running
60005
still running
70006
still running
80007
still running
90008
still running
100009
still running
110010
still running
120011
still running
130012
still running
140013
still running
150014
still running
160015
still running
170016
still running
180017
still running
190018
still running
200019
still running
210020
still running
220021
still running
230022
still running
240023
still running
250024
still running
260025
still running
270026
still running
280027
still running
290028
still running
300029
still running
310030
still running
320031
still running
330032
still running
340033
still running
350034
still running
360035
still running
370036
still running
380037
still running
390038
still running
400039
still running
410040
still running
420041
still running
430042
still running
440043
still running
450044
still running
460045
still running
470046
still running
480047
still running
4

In [7]:
df_qas = df.reset_index(drop=True)
df_qas

,ga:date,ga:dimension1,ga:eventCategory,ga:eventAction,ga:eventLabel,ga:segment,ga:uniqueEvents
0,(other),(other),(other),(other),(other),Sesiones con lead,3866
1,20210707,1000653101.1625702265,Home Funnel,0|,answer|home,Sesiones con lead,1
2,20210707,1000653101.1625702265,Home Funnel,step1|funnelHouseType,answer|chalet,Sesiones con lead,1
3,20210707,1000653101.1625702265,Home Funnel,step2|funnelFrequencyLiving,answer|primary,Sesiones con lead,1
4,20210707,1000653101.1625702265,Home Funnel,step3|funnelHouseLocation,answer|insideUrban,Sesiones con lead,1
...,...,...,...,...,...,...,...
999896,20211231,996922895.1640937004,Home Funnel,step5|funnelSecondaryAccess,answer|garden,Sesiones con lead,1
999897,20211231,996922895.1640937004,Home Funnel,step6|funnelGrilledWindows,answer|yes,Sesiones con lead,1
999898,20211231,996922895.1640937004,Home Funnel,step7|funnelPostCode,answer|yes,Sesiones con lead,1
999899,20211231,996922895.1640937004,Home Funnel,step8|funnelTimeInhabitedHouse,answer|more4hours,Sesiones con lead,1


In [8]:
df_qas.to_csv('data_files/scorer_ga_qas_by_cid.csv', index=False)